In [71]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
import numpy as np
import random
from sklearn.metrics import r2_score

In [72]:
# Existing dataset for regression by sklearn.datasets
X , y = load_diabetes(return_X_y=True)

In [73]:
X.shape,y.shape

((442, 10), (442,))

In [74]:
X_train , X_test , y_train , y_test = train_test_split( X , y ,test_size=0.2 , random_state = 42)

In [80]:
# Mini Batch Gradient Descent from the scratch
class MBRegressor:
    def __init__(self,batch_size,epochs = 100,learning_rate = 0.01):
        self.b_size = batch_size
        self.coef = None
        self.intercept = None
        self.epochs = epochs
        self.l = learning_rate
    
    def fit(self,X_train,y_train):
        #Instantiation of intercept and coeffecients
        self.coef = np.ones(X_train.shape[1]) # Column size
        self.intercept = 0 
        
        # Looping epochs time
        for i in range(self.epochs):
            for j in range(int(X_train.shape[0]/self.b_size)): #  we calculate the batch size by int(the no of rows / batch size).
                
                # Now calculating the intercept but here we have batch size 
                # So we will use np.sample to generate random batch size samples from the data. 
                
                # First we will fetch the ids for the nummber of random values from the dataset
                idx = random.sample(range(X_train.shape[0]),self.b_size) # generates row/b_size no of random ids from the domain

                # Now calculating yhat for calculation of intercept
                y_hat = np.dot(X_train[idx],self.coef) + self.intercept 

                # We need intercept derivaive acting as a loss function in calculation of new intercept value
                intercept_deri = -2 * np.mean(y_train[idx] - y_hat)

                # Now calculating new intercept value
                self.intercept = self.intercept - (self.l * intercept_deri)

                # Now lets calculate new coeffecient values
                # First again like intercept we need coef_deri :
                coef_deri = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])

                # Now calculating new coef value:
                self.coef = self.coef - (self.l * coef_deri)


        print(self.coef , self.intercept)

    def predict(self,X_test):
        return (np.dot(X_test,self.coef) + self.intercept )
        

In [ ]:
model = MBRegressor(batch_size=int(X_train.shape[0]/50)) # we are just taking this batch size for testing .

In [82]:
model.fit(X_train,y_train)

[  47.5966421  -165.26811717  468.64897147  308.02613012  -48.97527356
 -102.70523842 -208.28402096  140.8817889   352.36816343  126.84051853] 155.441430905509


In [83]:
y_pred = model.predict(X_test)

In [84]:
r2_score(y_test,y_pred)

0.46096770581855073

In [85]:
# Now we will use SGD regressor for this and implement it 
mbr = SGDRegressor( learning_rate='constant' , eta0=0.1 ) 
# We can tweak the eta and make other changes also by passing paramters here

In [90]:
# NOW there is no special method for MiniBatchRegression so we will use a loop and partial fit to make the model fit to data in each epoch

batch_size = 25 #Explicitly set for this example

for i in range(100): #epochs
    idx = random.sample(range(X_train.shape[0]),batch_size) # Same as our scratch implementaion
    mbr.partial_fit(X_train[idx],y_train[idx]) # Partially fits one by one

In [91]:
mbr.coef_,mbr.intercept_

(array([  48.66407653, -131.2370863 ,  440.48127934,  274.87093215,
         -35.34979179,  -80.52081532, -197.33974199,  140.85576794,
         325.43902735,  149.21428062]),
 array([155.69762581]))

In [92]:
y_pred_mbr = mbr.predict(X_test)

In [93]:
r2_score(y_test,y_pred_mbr)

0.4579601843518969